In [1]:
from ultralytics import YOLO
import torch
from torchvision.transforms import Compose, Resize, ToTensor
from torchvision import transforms
import cv2
import matplotlib.pyplot as plt

In [8]:
model = YOLO("yolo11n-seg.pt")

In [10]:
import cv2 as cv
img = cv.imread("dataset/test.jpg")
results = model(img)[0]
results.show()  # display to screen
results.save(filename="result.jpg")
# cv.imshow("Display window", img)
k = cv.waitKey(0) # Wait for a keystroke in the window


0: 480x640 18 apples, 155.6ms
Speed: 4.7ms preprocess, 155.6ms inference, 28.7ms postprocess per image at shape (1, 3, 480, 640)


In [ ]:
!pip install timm torch

In [ ]:
model_type = "DPT_Large"
midas = torch.hub.load("intel-isl/MiDaS", model_type)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

In [ ]:
custom_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((384, 384)),  # or any size multiple of 32
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

img = cv2.imread("dataset/test.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_tensor = custom_transform(img).unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")

with torch.no_grad():
    prediction = midas(input_tensor)

# Resize prediction to original image size
prediction = torch.nn.functional.interpolate(
    prediction.unsqueeze(1),
    size=(448,640),
    mode="bicubic",
    align_corners=False
).squeeze()
depth_map = prediction.cpu().numpy()
plt.imshow(depth_map)